In [9]:
from bokeh.io import show, output_notebook, curdoc
from bokeh.plotting import figure
from ipywidgets import interact
from bokeh.layouts import row, column, widgetbox, gridplot
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, TextInput, Button, Paragraph, Select

output_notebook()


Loading BokehJS ...

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime as dt
import altair as alt

In [11]:
data = pd.read_csv('us-counties.csv')
data

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0
...,...,...,...,...,...,...
95415,2020-04-27,Sublette,Wyoming,56035.0,1,0
95416,2020-04-27,Sweetwater,Wyoming,56037.0,10,0
95417,2020-04-27,Teton,Wyoming,56039.0,64,1
95418,2020-04-27,Uinta,Wyoming,56041.0,6,0


In [12]:
#create a new dataframe consisting only of date and cases
df_cases = data.loc[:, ['date', 'cases']]
df_cases.index = pd.to_datetime(df_cases.date)
df_cases = df_cases.drop(['date'], axis=1)

#create a new dataframe consisting only of date and deaths
df_deaths = data.loc[:, ['date', 'deaths']]
df_deaths.index = pd.to_datetime(df_deaths.date)
df_deaths = df_deaths.drop(['date'], axis=1)

#daily cumulative cases/deaths
cases_daily = df_cases.groupby(df_cases.index).sum()
deaths_daily = df_deaths.groupby(df_deaths.index).sum()

#weekly cumulative cases/deaths
df_c_weekly = cases_daily.loc[:, ['cases']]
df_c_weekly.index = df_c_weekly.index.week
df_c_weekly = df_c_weekly.reset_index()
cases_weekly = df_c_weekly.drop_duplicates(subset='date', keep='last')

df_d_weekly = deaths_daily.loc[:, ['deaths']]
df_d_weekly.index = df_d_weekly.index.week
df_d_weekly = df_d_weekly.reset_index()
deaths_weekly = df_d_weekly.drop_duplicates(subset='date', keep='last')

#monthly cumulative cases/deaths
df_c_monthly = cases_daily.loc[:, ['cases']]
df_c_monthly.index = df_c_monthly.index.month
df_c_monthly = df_c_monthly.reset_index()
cases_monthly = df_c_monthly.drop_duplicates(subset='date', keep='last')

df_d_monthly = deaths_daily.loc[:, ['deaths']]
df_d_monthly.index = df_d_monthly.index.month
df_d_monthly = df_d_monthly.reset_index()
deaths_monthly = df_d_monthly.drop_duplicates(subset='date', keep='last')

In [13]:
def modify_doc(doc):
    
    source_avg = ColumnDataSource(data=dict(x=cases_daily.index,
                                           y=cases_daily.cases))
    TOOLS = 'tap'
    
    plot = figure(plot_width=500, plot_height=400, tools=TOOLS, toolbar_location=None, title='COVID-19 in the US')
    
    plot.circle(x='x', y='y', size=5, color='blue', source=source_avg,
               hover_fill_color="black", fill_alpha=0.7, hover_alpha=1,
               line_color=None, hover_line_color="white")
    
    plot.line(x='x', y='y', source=source_avg, line_dash="4 4", line_width=1, color='gray')
    
    plot.xaxis.axis_label = "Time"
    plot.yaxis.axis_label = "Occurrences"
    
    #Set up widgets for input
    t = Paragraph()
    text = TextInput(title="Title", value='Cases')
    stat_select = Select(title="Cases/Deaths", value="Cases", options=["Cases", "Deaths"])
    time_select = Select(title="Periodicity", value="Daily", options=["Daily", "Weekly", "Monthly"])
    
    # Set up callbacks
    def update_title(attrname, old, new):
        t.text = text.value
    
    text.on_change('value', update_title)
    
    def update_data_source(attrname, old, new):
        #Get the current select value
        m = stat_select.value
        a = time_select.value
        
        #Generate the new source
        if (m=='Cases') & (a=='Daily'):
            t.text = "Cases"
            source_avg.data = dict(x=cases_daily.index, y=cases_daily.cases)
        elif (m=='Cases') & (a=='Weekly'):
            t.text = "Cases"
            source_avg.data = dict(x=cases_weekly.date, y=cases_weekly.cases)
        elif (m=='Cases') & (a=='Monthly'):
            t.text = "Cases"
            source_avg.data = dict(x=cases_monthly.date, y=cases_monthly.cases)
        elif (m=='Deaths') & (a=='Daily'):
            t.text = "Deaths"
            source_avg.data = dict(x=deaths_daily.index, y=deaths_daily.deaths)
        elif (m=='Deaths') & (a=='Weekly'):
            t.text = "Deaths"
            source_avg.data = dict(x=deaths_weekly.date, y=deaths_weekly.deaths)
        elif (m=='Deaths') & (a=='Monthly'):
            t.text = "Deaths"
            source_avg.data = dict(x=deaths_monthly.date, y=deaths_monthly.deaths)
            
            
    stat_select.on_change('value', update_data_source)
    time_select.on_change('value', update_data_source)
    
    # Set up layouts and add to document
    inputs = widgetbox(stat_select, time_select)
    t.text = "Cases"
    layout = column(t, row(inputs,plot, width=700))
    doc.add_root(layout)
    
show(modify_doc)